#Preparación y Clasificación de Texto con Redes Neuronales en TensorFlow/Keras
En el mundo del Procesamiento de Lenguaje Natural, las redes neuronales son herramientas increíblemente poderosas para comprender y generar texto. Sin embargo, antes de que podamos alimentar nuestro texto crudo a estos modelos, necesitamos transformarlo en un formato numérico que puedan entender. Este notebook te guiará a través de los pasos esenciales para preparar datos de texto y construir un modelo básico de clasificación con TensorFlow/Keras.

Aprenderemos sobre:

* Creación de un Dataset Sintético: Generaremos un pequeño conjunto de datos de ejemplo para ilustrar el proceso.
* Tokenización: Convertiremos palabras en números enteros, creando un vocabulario.
* Padding (Relleno): Aseguraremos que todas nuestras secuencias de texto tengan la misma longitud, un requisito clave para las redes neuronales.
* Construcción y Entrenamiento de un Modelo Simple: Crearemos una red neuronal con una capa de Embedding para aprender representaciones de palabras y un clasificador para asignar etiquetas.
* Predicción: Veremos cómo nuestro modelo puede clasificar un nuevo texto.

##1) Importación de Librerías Necesarias
Esta sección importa todas las bibliotecas de Python que utilizaremos en el notebook. pandas es fundamental para la manipulación de datos, mientras que tensorflow y sus módulos de preprocesamiento (Tokenizer, pad_sequences) y construcción de modelos (Sequential, Embedding, Dense, etc.) son el corazón de nuestro pipeline de PLN.

In [21]:
# Importar bibliotecas necesarias
import pandas as pd # Importa la librería pandas, esencial para trabajar con DataFrames (estructuras de datos tabulares).
import tensorflow as tf # Importa la librería TensorFlow, la base para construir y entrenar modelos de Machine Learning.
from tensorflow.keras.preprocessing.text import Tokenizer # Importa la clase Tokenizer para convertir texto en secuencias de números (tokens).
from tensorflow.keras.preprocessing.sequence import pad_sequences # Importa la función pad_sequences para asegurar que todas las secuencias de texto tengan la misma longitud.

##2) Creación y Guardado de un Conjunto de Datos Sintético
Aquí, creamos un pequeño dataset de ejemplo en Python. Consiste en oraciones de texto y sus etiquetas correspondientes (0 para "negativo" y 1 para "positivo"). Luego, guardamos este dataset en un archivo CSV, simulando un escenario real donde los datos se cargan desde un archivo externo.

In [22]:
# Crear un conjunto de datos sintético y guardarlo como CSV
# Ejemplos de texto y etiquetas
data = {
    "texto": [
        "No me gusta jugar a la play",
        "Me encanta los dias lluviosos",
        "Odio la sopa",
        "Me gusta mucho las semitas",
        "Prefiero café con leche que café solo",
        "La comida argentina es increíble",
        "Amo la electronica",
        "Los días soleados son feos",
        "Ver peliculas es un buen pasatiempo",
        "Me gustaria aprender a cocinar",
        "Es horrible el verano",
        "La plaza estaba limpia, pero el día estaba caluroso",
        "Se pasaron los fideos, pero salio rica la comida",
    ],
    "etiquetas": [
        0,  # Negativa
        1,  # Positiva
        0,  # Negativa
        1,  # Positiva
        1,  # Positiva
        1,  # Positiva
        1,  # Positiva
        0,  # Positiva
        1,  # Positiva
        1,  # Positiva
        0,  # Negativa
        0,  # Negativa
        1,  # Positiva
    ]
}

# Convertir a DataFrame
datos = pd.DataFrame(data)

# Guardar como CSV
datos.to_csv("text_dataset.csv", index=False)

print("Archivo 'text_dataset.csv' creado con éxito.")

Archivo 'text_dataset.csv' creado con éxito.


##3) Carga y Preprocesamiento del Conjunto de Datos de Texto
Esta es la fase crucial donde preparamos el texto para que una red neuronal lo entienda. Primero, cargamos el CSV que acabamos de crear. Luego, aplicamos la tokenización para convertir palabras en números, y el padding para que todas las secuencias resultantes tengan la misma longitud, lo cual es esencial para la entrada a las capas de las redes neuronales.

In [23]:
# Cargar el conjunto de datos de texto
datos = pd.read_csv("text_dataset.csv")

In [24]:
# Tokenización del texto
# Instancia un objeto Tokenizer. num_words=5000 significa que solo se considerarán
# las 5000 palabras más frecuentes en el vocabulario. Las palabras restantes serán ignoradas.
tokenizer = Tokenizer(num_words=5000)
# 'fit_on_texts' construye el vocabulario interno del tokenizer basado en los textos proporcionados.
# Asigna un ID numérico único a cada palabra.
tokenizer.fit_on_texts(datos["texto"])
# 'texts_to_sequences' convierte cada texto en una secuencia de IDs numéricos.
# Cada palabra se reemplaza por su ID correspondiente en el vocabulario.
secuencias = tokenizer.texts_to_sequences(datos["texto"])

# Padding (Relleno) de las secuencias
maxlen = 10  # Define la longitud máxima deseada para todas las secuencias.
# 'pad_sequences' rellena (o trunca) las secuencias para que todas tengan 'maxlen' elementos.
# Por defecto, el relleno se hace al principio (pre-padding) y con ceros (value=0).
X = pad_sequences(secuencias, maxlen=maxlen)
# Extrae las etiquetas (0 o 1) y las convierte a un array de NumPy para usarlas como salida del modelo.
y = datos["etiquetas"].values

# Imprimir las primeras secuencias numéricas y el dataset preprocesado para entender el resultado
print("\nPrimeras secuencias numéricas después de tokenización:")
for i, seq in enumerate(secuencias[:3]):
    print(f"Texto original: '{datos['texto'][i]}' -> Secuencia: {seq}")

print(f"\nPrimeras 3 secuencias después de padding (X, con maxlen={maxlen}):")
print(X[:3])
print(f"\nPrimeras 3 etiquetas (y): {y[:3]}")


Primeras secuencias numéricas después de tokenización:
Texto original: 'No me gusta jugar a la play' -> Secuencia: [12, 2, 5, 13, 6, 1, 14]
Texto original: 'Me encanta los dias lluviosos' -> Secuencia: [2, 15, 3, 16, 17]
Texto original: 'Odio la sopa' -> Secuencia: [18, 1, 19]

Primeras 3 secuencias después de padding (X, con maxlen=10):
[[ 0  0  0 12  2  5 13  6  1 14]
 [ 0  0  0  0  0  2 15  3 16 17]
 [ 0  0  0  0  0  0  0 18  1 19]]

Primeras 3 etiquetas (y): [0 1 0]


##4) Construcción y Compilación del Modelo de Clasificación
Esta sección define la arquitectura de nuestra red neuronal. Utilizamos un modelo Sequential de Keras, que es adecuado para pilas de capas simples. La capa clave aquí es la Embedding, que convierte los IDs de palabras en vectores densos y continuos, aprendiendo representaciones significativas de las palabras. Luego, se agregan capas densas para la clasificación.

In [25]:
# Cambiar el modelo a uno adecuado para texto
# Se define un modelo secuencial, que es una pila lineal de capas.
modelo = tf.keras.Sequential([
    # Capa de Embedding:
    # input_dim: Tamaño del vocabulario + 1 (por el 0 para padding y palabras OOV). Aquí 5000 palabras + 1.
    # output_dim: Dimensión del vector de embedding para cada palabra (cada palabra se representará como un vector de 64 números).
    # input_length: La longitud fija de las secuencias de entrada después del padding (maxlen).
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64, input_length=maxlen),
    # Capa GlobalAveragePooling1D:
    # Reduce la salida 3D de la capa Embedding (batch_size, maxlen, embedding_dim) a 2D (batch_size, embedding_dim).
    # Toma el promedio de los vectores de embedding a lo largo de la dimensión de la secuencia,
    # resumiendo la información de la oración en un solo vector.
    tf.keras.layers.GlobalAveragePooling1D(),
    # Capa Densa (Dense):
    # units=1: Una única neurona de salida para la clasificación binaria (positivo/negativo).
    # activation='sigmoid': Función de activación Sigmoide. Produce una probabilidad entre 0 y 1,
    # ideal para problemas de clasificación binaria (0 si < 0.5, 1 si >= 0.5).
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compilación del modelo
# Configura el proceso de aprendizaje del modelo.
modelo.compile(
    # optimizer: Algoritmo de optimización que ajusta los pesos del modelo. Adam es una buena opción por defecto.
    optimizer=tf.keras.optimizers.Adam(),
    # loss: Función de pérdida que el modelo intentará minimizar.
    # 'binary_crossentropy' es ideal para problemas de clasificación binaria donde la salida es una probabilidad.
    loss="binary_crossentropy",
    # metrics: Métricas para evaluar el rendimiento del modelo durante el entrenamiento y la evaluación.
    # 'accuracy' (precisión) mide la proporción de predicciones correctas.
    metrics=["accuracy"]
)

# Mostrar un resumen de la arquitectura del modelo
modelo.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##5) Entrenamiento del Modelo
En esta etapa, el modelo aprende de los datos preprocesados. La función fit() entrena la red neuronal durante un número específico de epochs (pasadas completas sobre todo el dataset) y con un batch_size (número de muestras procesadas antes de actualizar los pesos del modelo).

In [26]:
# Entrenamiento del modelo
# X: Las secuencias de texto preprocesadas (entradas).
# y: Las etiquetas correspondientes a cada secuencia (salidas esperadas).
# epochs=15: El número de veces que el modelo iterará sobre todo el conjunto de datos de entrenamiento.
# batch_size=2: El número de muestras que se procesarán antes de que los pesos del modelo se actualicen.
entrenamiento = modelo.fit(X, y, epochs=15, batch_size=2)

Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4097 - loss: 0.6945
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6825 - loss: 0.6879
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6616 - loss: 0.6786
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7491 - loss: 0.6677
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8387 - loss: 0.6480
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6929 - loss: 0.6531
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7210 - loss: 0.6398
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6616 - loss: 0.6417
Epoch 9/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6960 - loss: 0.6332
Epoch 10/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7918 - loss: 0.6081
Epoch 11/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7939 - loss: 0.5909
Epoch 12/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6564 - loss: 0.6171
E

##6) Predicción con un Nuevo Texto de Ejemplo
Finalmente, utilizamos nuestro modelo entrenado para hacer una predicción sobre un nuevo texto. Este texto también debe pasar por los mismos pasos de tokenización y padding que los datos de entrenamiento antes de ser alimentado al modelo.

In [27]:
# Predicción con ejemplo de texto
texto_ejemplo = "Desayunar café con leche y semitas me encanta" # Nuevo texto para predecir.

# Preprocesar el texto de ejemplo exactamente igual que los datos de entrenamiento:
# 1. Convertir el texto a una secuencia de IDs numéricos usando el mismo tokenizer.
secuencia_ejemplo = tokenizer.texts_to_sequences([texto_ejemplo])
# 2. Aplicar padding para que la secuencia tenga la misma longitud máxima (maxlen).
secuencia_ejemplo = pad_sequences(secuencia_ejemplo, maxlen=maxlen)

# Realizar la predicción usando el modelo entrenado.
# El modelo produce una probabilidad (debido a la activación sigmoide en la última capa).
prediccion = modelo.predict(secuencia_ejemplo)

# Imprimir el texto original y la probabilidad predicha.
# prediccion[0] accede al primer (y único) resultado de la predicción, ya que solo pasamos un texto.
print(f"Predicción para el texto '{texto_ejemplo}': {prediccion[0]}")

# Interpretar la predicción: si la probabilidad es > 0.5, se clasifica como positiva (1); de lo contrario, como negativa (0).
if prediccion[0] > 0.5:
    print(f"El modelo clasifica el texto como: Positivo (1)")
else:
    print(f"El modelo clasifica el texto como: Negativo (0)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicción para el texto 'Desayunar café con leche y semitas me encanta': [0.63094586]
El modelo clasifica el texto como: Positivo (1)


#Conclusiones
Este notebook ha sido una demostración práctica de cómo se preprocesa el texto para que sea comprensible para las redes neuronales y cómo se construye un modelo de clasificación de texto básico.

Hemos visto que:

* El texto debe ser numerizado: No podemos introducir palabras directamente en una red neuronal. La tokenización es el puente entre el texto y los números.
* La uniformidad es clave: Las redes neuronales requieren entradas de tamaño fijo. El padding resuelve este problema, aunque puede introducir ruido si las secuencias son muy cortas o truncar información si son muy largas.
* La capa Embedding es fundamental: Esta capa es la magia detrás de cómo las redes neuronales aprenden representaciones densas y significativas de las palabras, capturando relaciones semánticas y sintácticas en el proceso de entrenamiento.
* Modelos simples pueden aprender: Incluso con un dataset pequeño y una arquitectura sencilla (Embedding + GlobalAveragePooling1D + Dense), el modelo puede aprender a clasificar textos con una precisión razonable, demostrando el poder de las redes neuronales en PLN.

Para aplicaciones del mundo real, se necesitarían datasets mucho más grandes, arquitecturas de modelos más complejas (como RNNs o Transformers), y técnicas de preprocesamiento más avanzadas. Sin embargo, los conceptos de tokenización, padding y embeddings que exploramos aquí forman la base de casi todas las aplicaciones de PLN modernas con redes neuronales.